In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pyfluids as pf
from scipy.optimize import brentq

In [4]:
WATER = pf.Fluid(pf.FluidsList.Water)

In [65]:
# Ett system MED matarvattenförvärmning
# 7 olika tillstånd
#   1 - Precis efter kondensator - Mättad vätska. P1 = P7, S1 = S7
#   2 - Precis efter pump 1 - Trycksatt vätska. P2 = P3 = P6, S1 = S2
#   3 - Precis efter matarvattenförvärmning. # P2 = P3 = P6
#   4 - Precis efter pump 2. # P4 = P5, S3 = S4
#   5 - Precis efter ångpanna. # P4 = P5, T5
#   6 - Precis efter turbin fraktion y. # S5 = S6 = S7
#   7 - Precis efter turbin fraktion 1-y. # S5 = S6 = S7

P4 = P5 = 13*1e6 # Pa
T5 = 600 # C 
WATER.update(pf.Input.pressure(P5), pf.Input.temperature(T5))

H5 = WATER.enthalpy # J/(kg)
S5 = WATER.entropy # J/(kg*K)
print("H5: ", H5)
print("S5: ", S5)

H5:  3600365.151558058
S5:  6760.92808000537


In [123]:
# Givet från förra uppgiften:
#   Kondensatortryck:
P1 = P7 = 28030.389667251104 # Pa

WATER.update(pf.Input.pressure(P7),  pf.Input.quality(85))
H7 = WATER.enthalpy
S7 = WATER.entropy
T7 = WATER.temperature
print("H7: ", H7)
print("S7: ", S7)
print("T7: ", T7)

H7:  2271016.354096498
S7:  6760.92808000537
T7:  67.54255917304096


In [ ]:
# Tillstånd 1 har samma tryck som tillstånd 7 och ångkvalité 0% (mättad vätska)
WATER.update(pf.Input.pressure(P7), pf.Input.quality(0))
H1 = WATER.enthalpy
S1 = WATER.entropy
T1 = WATER.temperature
print("H1: ", H1)
print("S1: ", S1)
print("T1: ", T1)

H1:  282767.099526563
S1:  925.023534875328
T1:  67.54255917304096


In [127]:
# SÖKES: tryck i tillstånd 2, 3, 6
#   Vi vet att tillstånd 3 är mättad vätska (ångkvalité 0%) och samma tryck som tillstånd 6.
#   Vi vet att tillstånd 6 är överhettad ånga

def f(P, y): 
    # andel = fraktion y från turbin
    WATER.update(pf.Input.pressure(P), pf.Input.entropy(S1))
    H2 = WATER.enthalpy

    WATER.update(pf.Input.pressure(P), pf.Input.quality(0))
    H3 = WATER.enthalpy

    WATER.update(pf.Input.pressure(P), pf.Input.entropy(S7))
    H6 = WATER.enthalpy

    #print("Percentage: ", y)
    #print("H2: ", H2)
    #print("H3: ", H3)
    #print("H6: ", H6)
    #print("---")
    # Energiprincipen
    return ((1-y)*H2 + y*H6) - H3

def show_graph():
    percentage = np.linspace(0, 1, 101)
    x = np.linspace(1000, 1e7, 101)
    y = np.zeros(shape=[len(percentage), len(x)])
    for index1, percent in enumerate(percentage):
        #print("new percentage: ", percent)
        for index2, pressure in enumerate(x):
            y[index1, index2] = f(pressure, percent)
    #print(y)
    plt.plot(x.T, y.T)

    plt.xlabel("Pressure")
    plt.ylabel("Energy ")
    plt.show()
#show_graph()

percentages = np.linspace(0, 1, 500)
# Create array of possible pressure values for different fractions y 
sol = np.zeros([len(percentages),2])

for index, val in enumerate(percentages):
    try:
        sol[index,:] = (val, brentq(f,1000,P5, args=val))
    except:
        pass
# Solution stores many rows of two columns (fraction y, pressure)
sol = sol[sol[:, 1] > 0]

S3 = np.zeros_like(sol[:,1])
H3 = np.zeros_like(sol[:,1])
for index, row in enumerate(sol):
    P = row[1]
    if P == 0:
        continue
    WATER.update(pf.Input.pressure(P), pf.Input.quality(0))
    S3[index] = WATER.entropy
    H3[index] = WATER.enthalpy

S2 = np.zeros_like(sol[:,1])
H2 = np.zeros_like(sol[:,1])
for index, row in enumerate(sol):
    P = row[1]
    if P == 0:
        continue
    WATER.update(pf.Input.pressure(P), pf.Input.entropy(S1))
    S2[index] = WATER.entropy
    H2[index] = WATER.enthalpy

S6 = np.zeros_like(sol[:,1])
H6 = np.zeros_like(sol[:,1])
for index, row in enumerate(sol):
    P = row[1]
    if P == 0:
        continue
    WATER.update(pf.Input.pressure(P), pf.Input.entropy(S5))
    S6[index] = WATER.entropy
    H6[index] = WATER.enthalpy

print(H3)

[ 282767.09952656  286763.93361168  290783.18449788  294824.9693912
  298889.40536166  302976.60932073  307086.69799813  311219.78791771
  315375.99537321  319555.43640269  323758.22676275  327984.4819019
  332234.31693319  336507.8466063   340805.18527865  345126.44688631
  349471.74491368  353841.19236284  358234.90172185  362652.98493304
  367095.55335947  371562.71775187  376054.58821423  380571.27416864
  385112.8843199   389679.52661899  394271.30822602  398888.33547262
  403530.7138234   408198.54783705  412891.9411265   417610.99631834
  422355.8150121   427126.49773835  431923.14391638  436745.85181122
  441594.71849013  446469.83977827  451371.31021408  456299.22300372
  461253.66997519  466234.74153178  471242.52660506  476277.11260719
  481338.58538276  486427.02916026  491542.52650283  496685.15825875
  501855.00351121  507052.1395281   512276.64171049  517528.58354198
  522808.03653658  528115.07018656  533449.75191017  538812.1469988
  544202.31856392  549620.32748363  5

In [129]:
# Tillstånd 4 har samma entropi som tillstånd 3 och samma tryck som tillstånd
H4 = np.empty_like(S3)
for index, entropy in enumerate(S3):
    WATER.update(pf.Input.entropy(entropy), pf.Input.pressure(P4))
    H4[index] = WATER.enthalpy 
#print(H4)

In [137]:
W_pump1 = H2 - H1
W_pump2 = H4 - H3
Q_in = H5 - H4

W_turb = np.empty_like(sol)
for index, row in enumerate(sol):
    y = row[0]
    #FIXME: 
    W_turb[index] = (y, H5 - H6[index] +(1-y)*(H6[index] - H7))

#print(W_turb[:,1])
W_net = W_turb[:,1] - W_pump1 - W_pump2
verkningsgrad = np.divide(W_net, Q_in)
max_index = np.argmax(verkningsgrad)

#print("W_net: ", W_net)
#print("Q_in: ", Q_in)
#print("verkningsgrad: ",verkningsgrad)
print(f"maximala verkningsgraden fås vid y={sol[max_index,0]*100}% då verkningsgraden är {np.max(verkningsgrad)} ")

maximala verkningsgraden fås vid y=20.841683366733466% då verkningsgraden är 0.42728974233087 
